# Data Cleaning

In [7]:
# This is Ayaan Cleaning legit code

import pandas as pd
from collections import defaultdict

# Load the DataFrame
df = pd.read_csv('new_amplitude_export_2025_chunk_0_18155.csv')

# Drop rows with NaN values in the 'region' column
df = df.dropna(subset=['region'])

#print(df.head())

# Print unique regions
print(set(df['region']))

# Group rows by 'session_id'
mp = defaultdict(list)
for i, sid in enumerate(df['session_id']):  # Iterate over the values in the 'session_id' column
  mp[sid].append(df.iloc[i])

# Print the grouped data
# print(mp)

# mp = defaultdict(list)
# for i, sid in enumerate(df['session_id']):
#   mp[sid].append(df.iloc[i])

  
for keys in mp.keys():
  mp[keys].sort(key = lambda x: x['client_event_time'])
  print(list(map(lambda x: x['event_type'], mp[keys])))

{'Tennessee', 'Mexico City', 'Michigan', 'Colorado', 'New York', 'Montana', 'Virginia', 'Georgia', 'Alabama', 'Maharashtra', 'North Carolina', 'Massachusetts', 'California', 'Oregon', 'Ontario', 'South Carolina', 'Washington', 'Texas', 'Ohio', 'Pennsylvania', 'New Jersey', 'Illinois', 'Connecticut', 'Missouri', 'Kansas', 'Lower Silesia', 'Utah', 'Indiana', 'Florida', 'Iowa', 'None', 'Arizona'}
['session_end']
['application-window-opened', 'dashboard:my-book::view', 'dashboard:my-book:layout:render', 'dashboard:my-book:widget:render', 'dashboard:my-book:widget:render', 'dashboard:my-book:widget:render', 'dashboard:my-book:configurable-table:render', 'dashboard:my-book:configurable-table:render', 'triaged-submission-list:my-book::view', 'triaged-submission-list:my-book:configurable-table:render', 'session_start', 'application-window-opened', 'dashboard:my-book:layout:render', 'dashboard:my-book::view', 'dashboard:my-book:widget:render', 'dashboard:my-book:widget:render', 'dashboard:my-bo

In [2]:
import pandas as pd

# Load CSV into a DataFrame
df = pd.read_csv('new_amplitude_export_2025_chunk_0_18155.csv')

# Convert timestamps to datetime format
df['event_time'] = pd.to_datetime(df['event_time'])
df['client_upload_time'] = pd.to_datetime(df['client_upload_time'])

# Sort by session_id and event_time before grouping
df = df.sort_values(by=['session_id', 'event_time'])

# Group by session_id and calculate session duration
session_durations = df.groupby('session_id').agg(
    start_time=('event_time', 'first'),
    end_time=('event_time', 'last')
)

# Compute the session duration
session_durations['duration'] = session_durations['end_time'] - session_durations['start_time']

# Filter out sessions with zero duration (single-event sessions)
session_durations = session_durations[session_durations['duration'] > pd.Timedelta(0)]

# Sort by session duration (shortest to longest)
sorted_sessions = session_durations.sort_values(by='duration').reset_index()

# Display DataFrame
print(sorted_sessions)


        session_id              start_time                end_time  \
0    1737969111172 2025-01-27 09:11:51.172 2025-01-27 09:11:51.173   
1    1737503477356 2025-01-21 23:51:17.356 2025-01-21 23:51:17.357   
2    1737579323610 2025-01-22 20:55:23.610 2025-01-22 20:55:23.611   
3    1737503466424 2025-01-21 23:51:06.424 2025-01-21 23:51:06.425   
4    1737503466420 2025-01-21 23:51:06.420 2025-01-21 23:51:06.421   
..             ...                     ...                     ...   
671  1737961202124 2025-01-27 09:00:24.107 2025-01-27 09:58:16.139   
672  1737575627959 2025-01-22 20:00:52.293 2025-01-22 20:58:47.319   
673  1737960409163 2025-01-27 09:00:02.166 2025-01-27 09:59:04.734   
674  1737569917280 2025-01-22 20:00:15.974 2025-01-22 20:59:30.276   
675  1737576016578 2025-01-22 20:00:16.578 2025-01-22 20:59:55.919   

                  duration  
0   0 days 00:00:00.001000  
1   0 days 00:00:00.001000  
2   0 days 00:00:00.001000  
3   0 days 00:00:00.001000  
4   0 days 00:

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import os

# Folder containing all CSV files
data_folder = "data"

# # Load all CSVs into a single DataFrame
# all_dfs = []
# for file in os.listdir(data_folder):
#     if file.endswith(".csv"):
#         file_path = os.path.join(data_folder, file)
#         df = pd.read_csv(file_path)
#         df['event_time'] = pd.to_datetime(df['event_time'])
#         all_dfs.append(df)

# # Merge all CSVs into one DataFrame
# df = pd.concat(all_dfs, ignore_index=True)


df = pd.read_csv('new_amplitude_export_2025_chunk_0_18155.csv')

# Drop rows with NaN values in the 'region' column
df = df.dropna(subset=['region'])




# Convert event_time to datetime and sort
df = df.sort_values(by=['session_id', 'event_time'])

# Feature Engineering
df['session_duration'] = df.groupby('session_id')['event_time'].transform(lambda x: (x.max() - x.min()).seconds)
df['time_since_last_event'] = df.groupby('session_id')['event_time'].diff().dt.seconds.fillna(0)
df['event_count'] = df.groupby('session_id').cumcount() + 1  # Event count in session
df['hour_of_day'] = df['event_time'].dt.hour  # Time of day

# Encode categorical variables
le = LabelEncoder()
df['event_type_encoded'] = le.fit_transform(df['event_type'])

# Define target as the next event
df['next_event'] = df.groupby('session_id')['event_type_encoded'].shift(-1)
df = df.dropna()

# **Filter Out Classes with Less Than 2 Occurrences**
event_counts = df['next_event'].value_counts()
valid_classes = event_counts[event_counts >= 2].index
df = df[df['next_event'].isin(valid_classes)]  # Keep only frequent classes

# Features & Labels
features = ['event_type_encoded', 'session_duration', 'time_since_last_event', 'event_count', 'hour_of_day']
X = df[features]
y = df['next_event']

# Train-test split (without stratify to avoid errors)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Improved Random Forest Model
clf = RandomForestClassifier(
    n_estimators=500,  # Increased trees
    max_depth=15,  # Prevent overfitting
    min_samples_split=5,  # Reduce variance
    random_state=42
)
clf.fit(X_train, y_train)

# Evaluate model
accuracy = clf.score(X_test, y_test)
print(f"Improved Model Accuracy: {accuracy * 100:.2f}%")

# Function to recommend action
def recommend_action(event_sequence):
    if not event_sequence:
        return "No events given"
    
    last_event = event_sequence[-1]
    
    # Handle unseen labels
    if last_event not in le.classes_:
        print(f"Warning: Event '{last_event}' not in training data. Assuming similar event.")
        return "Unknown event"

    encoded_seq = le.transform([last_event])  # Encode last event
    
    # Default values for missing session info
    input_data = pd.DataFrame([[encoded_seq[0], 30, 5, 5, 14]], columns=features)

    predicted_next = clf.predict(input_data)[0]  # Predict next event
    predicted_event = le.inverse_transform([predicted_next])[0]  # Convert back to event name
    
    # If model predicts session_end, trigger recommendation
    if predicted_event == "session_end":
        return "Recommend now!"
    return f"Next likely event: {predicted_event}"

# Example usage
user_events = ['page_view', 'add_to_cart', 'checkout']
print(recommend_action(user_events))


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [5]:
# from collections import defaultdict

# df = pd.read_csv('new_amplitude_export_2025_chunk_0_18155.csv')
# df['event_time'] = pd.to_datetime(df['event_time'])
# df = df.sort_values(by=['session_id', 'event_time'])

# #Group events by session to track user paths
# paths = defaultdict(list)
# for session_id, group in df.groupby('session_id'):
#     paths[session_id] = group['event_type'].tolist()

# #Count feature transitions (e.g., A -> B)
# transition_counts = defaultdict(lambda: defaultdict(int))
# for path in paths.values():
#     for i in range(len(path) - 1):
#         transition_counts[path[i]][path[i + 1]] += 1

# #Convert counts to transition probabilities (percentages)
# transition_matrix = []
# for state, transitions in transition_counts.items():
#     total = sum(transitions.values())
#     for next_state, count in transitions.items():
#         probability = round((count / total) * 100, 2)
#         transition_matrix.append((f"{state} -> {next_state}", probability))

# #Convert to DataFrame
# transition_df = pd.DataFrame(transition_matrix, columns=["Transition", "Probability"])
# transition_df = transition_df.sort_values(by="Transition").reset_index(drop=True)

# pd.set_option('display.float_format', '{:.2f}%'.format)
# print("Transition Matrix (Percentages):\n")
# print(transition_df.to_string(index=False, col_space=0, justify='left'))

In [10]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from collections import Counter

# Load dataset
df = pd.read_csv("new_amplitude_export_2025_chunk_0_18155.csv")

# Ensure necessary columns exist
if 'session_id' not in df.columns or 'event' not in df.columns:
    raise ValueError("Dataset must contain 'session_id' and 'event' columns.")

# Encode categorical labels
label_encoder = LabelEncoder()
df["event"] = label_encoder.fit_transform(df["event"])

# Group events by session
session_data = df.groupby("session_id")["event"].apply(list)
session_lengths = df.groupby("session_id")["event"].count()

# Encode sequences with padding
MAX_SEQUENCE_LENGTH = 13  # Fixed sequence length

def encode_sequence(seq, session_length):
    """Encodes a sequence with padding and appends session length."""
    encoded = [event for event in seq]  # Already encoded by LabelEncoder
    padded_sequence = [0] * (MAX_SEQUENCE_LENGTH - len(encoded)) + encoded  # Pad to fixed length
    return np.array(padded_sequence + [session_length], dtype=np.float32)  # Convert to float32

# Prepare features (X) and labels (y)
X = np.array([encode_sequence(seq, length) for seq, length in zip(session_data, session_lengths)], dtype=np.float32)
y = np.array(df.groupby("session_id")["label"].first(), dtype=np.int32)  # Ensure y is integer-encoded

# Handle rare classes (remove classes with fewer than 2 occurrences)
counts = Counter(y)
rare_classes = {cls for cls, count in counts.items() if count < 2}
valid_indices = [i for i in range(len(y)) if y[i] not in rare_classes]

X, y = X[valid_indices], y[valid_indices]  # Filter valid instances

# Normalize X (optional but helps training)
X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-6)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost Classifier with optimized parameters
clf = XGBClassifier(
    n_estimators=800,
    learning_rate=0.01,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0.1,
    reg_lambda=1.5,
    random_state=42
)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.2%}")


ValueError: Dataset must contain 'session_id' and 'event' columns.